# Implementar un MLP con PyTorch para clasificación basado en el dataset de agresividad
# Uso de word embeddings como representación de datos


<img src="figs/fig-diagrama-clasificador2.png" width="900">


### 1. **Representar los datos en el modelo de word embeddings seleccionado**:  
   - #### Generalmente, solo se tokeniza para separar adecuadamente las palabras.
   - #### Sin embargo, dependiendo del modelos de word embeddings algunos preprocesamientos puede mejorar la representación.
   - #### Por ejemplo: 
      - ##### tokenizar y separar correctamente las oraciones y palabras
      - ##### convertir a minúsculas
      - ##### quitar acentos (dependiendo de la fuente de datos con la que se generaros los embeddings)
      - ##### quitar números y puntuación 


### 3. **Convertir los datos a vectores densos: word embeddings**:  
   - #### En el caso de textos corto a nivel de oración, un vector denso por oración. 

### 4. **Separar los datos para entrenamiento, validación y prueba**:  
   - #### Crear los dataset  con la función train_test_split 
   
### 5. **Definir la arquitectura de la red**:  
   - Definir una red de 2 capas, con funciones PReLU en las capas ocultas y una capa de salida

### 6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
### 7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   


# Cargar los datos de entrenamiento

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import fasttext

# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset = pd.read_json("./data/data_aggressiveness_es.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y = dataset['klass'].to_numpy()



# Cargar el modelo de Word Embeddings y crear los vectores densos

In [7]:
# Cargar el modelo de word embeddings
ft = fasttext.load_model('/Users/uacm/UACM_local/datos/MX.bin')


# Crear los vectores densos para cada texto. Se espera una oración corta

In [ ]:
# Extracción de los textos en arreglos de numpy
# Se aplica una función lambda para obtener el vector de cada texto del conjunto de datos
# El resultado es una nueva columna "embedding"
dataset["embedding"] = dataset["text"].map(lambda x: ft.get_sentence_vector(x))
dataset.head()

# Crear la matriz de datos para el entrenamiento, validación y prueba del modelo de la red neuronal

In [ ]:
# Cada renglón representa un documento codificado en un texto de embeddings
X = np.vstack(dataset['embedding'].to_numpy())
Y = dataset['klass'].to_numpy()

print("Datos:", X.shape) 
print("Etiquetas:", Y.shape) 



# Codificar las etiquetas

In [ ]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


# Preparar los conjuntos de datos  para el entrenamiento, validación y prueba

In [11]:
# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)

# Dividir el conjunto de entrenamiento en:  entrenamiento (90%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=42)



# Codificar las clases en forma one-hot 
NUM_CLASSES = 2
Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()

# Agregar dimensión de canales

X_train = torch.tensor(X_train).unsqueeze(1)  # (3694, 1, 300)
X_val = torch.tensor(X_val).unsqueeze(1)      # Similar para validación
X_test = torch.tensor(X_test).unsqueeze(1)    # Similar para prueba


# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader


In [ ]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape, Y_train_one_hot.shape

In [ ]:
Y_train_one_hot[:5],  Y_train_one_hot.shape

In [ ]:
np.unique(Y_train, return_counts=True)

In [ ]:
X_train.shape[1]

# Definición de la arquitectura de la red

In [35]:
import torch
import numpy as np

class Conv1DTextClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        padding=1
        kernel_size =3
        stride = 1
        dilation=1
        self.conv1 = nn.Conv1d(in_channels, out_channels=8, kernel_size=kernel_size, stride= stride, padding=padding, dilation=dilation)
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=4, kernel_size=kernel_size, padding=padding, dilation=dilation)

        L1 = ((300 + 2*padding - dilation * (kernel_size-1) -1 ) // stride ) + 1
        L2 = ((L1 + 2*padding - dilation * (kernel_size-1) -1 ) // stride ) + 1

        self.fc1 = nn.Linear(4 * L2, 8)  # Ajustar dimensiones según salida
        self.fc2 = nn.Linear(8, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Aplanar
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    


    

In [ ]:
class_counts = np.bincount(Y_train)  # Conteo de ejemplos por clase
print(class_counts)
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float32)
class_weights


# Entrenamiento de la red

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Establecer los parámetros de la red

# Parámetros de la red
input_size =  X_train.shape[1]
in_channels = 1
output_size = 2   # 2 clases

epochs = 10 # variar el número de épocas, para probar que funciona la programación 
                 # solo usar 2 épocas, para entrenamiento total usar por ejemplo 1000 épocas
learning_rate = 0.001 # Generalmente se usan learning rate pequeños (0.001), 

# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 16 # definir el tamaño del lote de procesamiento 


# Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

# X_train_t = torch.from_numpy(X_train)
# X_train_t = X_train_t.to(torch.float32)
X_train_t = X_train
Y_train_t = Y_train_one_hot

# X_val_t = torch.from_numpy(X_val)
# X_val_t = X_val_t.to(torch.float32)
X_val_t = X_val


# Crear la red
model = Conv1DTextClassifier(in_channels, output_size)

# Definir la función de pérdida
# Entropía Cruzada 
class_counts = np.bincount(Y_train)  # Conteo de ejemplos por clase
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float32)

# class_weights ayuda a dar mayor peso a las clases desbalanceadas.

criterion = nn.CrossEntropyLoss(weight=class_weights)
# Peso de las clases uniforme
# criterion = nn.CrossEntropyLoss() 

# Definir el optimizador
#Parámetros del optimizador: parámetros del modelo y learning rate 
# Adaptive Moment Estimation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")


for epoch in range(epochs):
# Poner el modelo en modo de entrenamiento
    model.train()  
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_train_t, Y_train_t, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = model(X_tr)  #invoca al método forward de la clase MLP
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente:
        #  
        optimizer.step()
        if np.random.random() < 0.1:
            print(f"Batch Error : {loss.item()}")

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")
    
    # Evalúa el modelo con el conjunto de validación
    model.eval()  # Establecer el modo del modelo a "evaluación"
    with torch.no_grad():  # No  calcular gradientes 
        y_pred = model(X_val_t)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)
        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_pred, average='macro'))
        print("R=", recall_score(Y_val, y_pred, average='macro'))
        print("F1=", f1_score(Y_val, y_pred, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred))


### Modo para predicción de datos

In [ ]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de 
# representación vectorial que el modelo entrenado, es decir, al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

# X_t = torch.from_numpy(X_test)
# X_t = X_t.to(torch.float32)
X_t = X_test

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
model.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test= model(X_t)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test = torch.argmax(y_pred_test, dim=1)

print(y_pred_test)


### Evaluación

In [ ]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


print(confusion_matrix(Y_test, y_pred_test))
print(classification_report(Y_test, y_pred_test, digits=4, zero_division='warn'))


### Evaluación de datos nuevos

In [ ]:

x_datos = ["Esa hija de la se llevó mis cosas", "ese hijo de se llevo el dinero", "mi app de calendario no sirve"]
# Transformar los datos a vectores densos: word embeddings
x_datos = [ft.get_sentence_vector(texto) for texto in x_datos]



# Apilar los vectores verticalmente para tener un ejemplo.
vectores = np.vstack(x_datos)

vectores = torch.tensor(vectores).unsqueeze(1)      # Similar para validación
print(vectores.shape)
x_datos_t = vectores

model.eval()  # Establecer el modo del modelo a "evaluación"
with torch.no_grad():  # No  calcular gradientes 
    y_pred = model(x_datos_t)
    y_pred = torch.argmax(y_pred, dim=1)
    print(le.inverse_transform(y_pred))

# Ejercicio 1

In [ ]:
# TODO: Modificar el número de neuronas de las capas lineales: capa1 y capa2 y evaluar si el modelo mejora o empeora.


# Ejercicio 2

In [ ]:
# TODO: Aplicar una función lamba para que preprocese los textos con los siguientes características:
#   - Tokenizar los datos para separar términos que estuvieran juntos por puntuación o símbolos extraños (sugerencia: usar word_tokenizer)
#   - convertir a minúsculas
#   - eliminar puntuación y símbolos duplicados
# Comprobar si el rendimiento del modelo mejora o empeora.
